Wrangling data with sql


In [ ]:
import sqlite3

import pandas as pd
from IPython.display import VimeoVideo

Prepare data


Connect


In [ ]:
#Run the cell below to connect to the nepal.sqlite database
%load_ext sql
%sql sqlite:////home/jovyan/nepal.sqlite

Explore

In [ ]:
#How many tables are in the nepal.sqlite database? What information do they hold
%%sql
SELECT *
FROM sqlite_schema

In [ ]:
#Select the name column from the sqlite_schema table, showing only rows where the type is "table"
%%sql
SELECT name
FROM sqlite_schema
WHERE type= "table"

In [ ]:
#Select all columns from the id_map table, limiting your results to the first five rows
%%sql
SELECT *
FROM id_map
LIMIT 5

In [ ]:
# How many observations are in the id_map table? Use the count command to find out.
%%sql
SELECT count(*)
from id_map

In [ ]:
# What districts are represented in the id_map table? Use the distinct command to determine the unique values in the district_id column
%%sql
SELECT distinct(district_id)
from id_map

In [ ]:
#How many buildings are there in id_map table? Combine the count and distinct commands to calculate the number of unique values in building_id
%%sql
SELECT count(distinct(building_id))
from id_map

In [ ]:
# For our model, we'll focus on Gorkha (district 4). Select all columns that from id_map, showing only rows where the district_id is 4 and limiting your results to the first five rows
%%sql
SELECT *
FROM id_map
where district_id = 4
limit 5

In [ ]:
#How many observations in the id_map table come from Gorkha? Use the count and WHERE commands together to calculate the answer
%%sql
SELECT COUNT(*)
FROM id_map
where district_id=4

In [ ]:
#How many buildings in the id_map table are in Gorkha? Combine the count and distinct commands to calculate the number of unique values in building_id, considering only rows where the district_id is 4
%%sql
SELECT count (distinct(building_id)) AS unique_building_gorkha
FROM id_map
where district_id=4

In [ ]:
# Select all the columns from the building_structure table, and limit your results to the first five rows
%%sql
SELECT *
FROM building_structure 
LIMIT 5

In [ ]:
#How many building are there in the building_structure table? Use the count command to find out
%%sql
SELECT count(*)
FROM building_structure

In [ ]:
#There are over 200,000 buildings in the building_structure table, but how can we retrieve only buildings that are in Gorkha? Use the JOIN command to join the id_map and building_structure tables, showing only buildings where district_id is 4 and limiting your results to the first five rows of the new table.
%%sql
SELECT *
FROM id_map
JOIN building_structure ON id_map.building_id=building_structure.building_id
WHERE district_id =4
LIMIT 5

In [ ]:
%%sql
SELECT  *
FROM id_map AS i
JOIN building_structure AS s ON i.building_id=s.building_id
WHERE district_id =4
LIMIT 5

In [ ]:
#Use the distinct command to create a column with all unique building IDs in the id_map table. JOIN this column with all the columns from the building_structure table, showing only buildings where district_id is 4 and limiting your results to the first five rows of the new table
%%sql
SELECT distinct(i.building_id),
        s.*
FROM id_map AS i
JOIN building_structure AS s ON i.building_id=s.building_id
WHERE district_id =4
LIMIT 5

In [ ]:
#How can combine all three tables? Using the query you created in the last task as a foundation, include the damage_grade column to your table by adding a second JOIN for the building_damage table. Be sure to limit your results to the first five rows of the new table
%%sql
SELECT distinct(i.building_id) AS b_id,
        s.*,
        d.damage_grade
FROM id_map AS i
JOIN building_structure AS s ON i.building_id=s.building_id
JOIN building_damage AS d ON i.building_id=d.building_id
WHERE district_id =4
LIMIT 5

In [ ]:
#Use the connect method from the sqlite3 library to connect to the database. Remember that the database is located at "/home/jovyan/nepal.sqlite"
conn = sqlite3.connect("/home/jovyan/nepal.sqlite")

In [ ]:
#Put your last SQL query into a string and assign it to the variable query
query = """
SELECT distinct(i.building_id) AS b_id,
        s.*,
        d.damage_grade
FROM id_map AS i
JOIN building_structure AS s ON i.building_id=s.building_id
JOIN building_damage AS d ON i.building_id=d.building_id
WHERE district_id =4
"""
print(query)

In [ ]:
#Use the read_sql from the pandas library to create a DataFrame from your query. Be sure that the building_id is set as your index column
df = pd.read_sql(query, conn, index_col= "b_id")

df.head()